In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import datasets
sns.set(style='white')

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp

tfk = tf.keras
tfkl = tf.keras.layers
tfd = tfp.distributions
tfpl = tfp.layers

2024-06-06 12:34:31.749045: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
encoded_size = 16
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)

In [4]:
input_shape = (28, 28, 1)
base_depth = 32

In [7]:
tfk.Sequential([
    tfkl.InputLayer(input_shape=input_shape),
    tfkl.Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
    tfkl.Conv2D(base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(4 * encoded_size, 7, strides=1,
                padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Flatten(),
    tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size), activation=None),
    tfpl.MultivariateNormalTriL(encoded_size, activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=1.0))
])

In [21]:
from collections import Sequence

ImportError: cannot import name 'Sequence' from 'collections' (/home/ww347/.conda/envs/google-env/lib/python3.10/collections/__init__.py)

In [22]:
print(tf.__version__)  # Should match with tfp.__version__
print(tfp.__version__)

2.11.0
0.14.0


In [19]:

# This code gives an Attribute error
# AttributeError: Exception encountered when calling layer "multivariate_normal_tri_l" (type MultivariateNormalTriL).

# module 'collections' has no attribute 'Sequence'

# Call arguments received by layer "multivariate_normal_tri_l" (type MultivariateNormalTriL):
#   • inputs=tf.Tensor(shape=(None, 152), dtype=float32)
#   • args=<class 'inspect._empty'>
#   • kwargs={'training': 'None'}
    

Here received another odd error; though at least the packages are now loading!

In [11]:
import pprint

In [8]:
tfd.LKJ?

Init signature:
tfd.LKJ(
    dimension,
    concentration,
    input_output_cholesky=False,
    validate_args=False,
    allow_nan_stats=True,
    name='LKJ',
)
Docstring:     
The LKJ distribution on correlation matrices.

This is a one-parameter family of distributions on correlation matrices.  The
probability density is proportional to the determinant raised to the power of
the parameter: `pdf(X; eta) = Z(eta) * det(X) ** (eta - 1)`, where `Z(eta)` is
a normalization constant.  The uniform distribution on correlation matrices is
the special case `eta = 1`.

The distribution is named after Lewandowski, Kurowicka, and Joe, who gave a
sampler for the distribution in [(Lewandowski, Kurowicka, Joe, 2009)][1].

Note: For better numerical stability, it is recommended that you use
`CholeskyLKJ` instead.

#### Examples

```python
# Initialize a single 3x3 LKJ with concentration parameter 1.5
dist = tfp.distributions.LKJ(dimension=3, concentration=1.5)

# Evaluate this at a batch of two obser